#### SCCAF clustering for Cai 2020 + 2022 healthy PBMCs

**Objective**: Run SCCAF clustering in separate environment to solve dependency issues

- **Developed by**: Mairi McClean
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- v230316

### Use `SCCAF` to select `leiden` resolution

> What is SCCAF? "Single Cell Clustering Assessment Framework"
>> It is a way to cluster cells based on gene expression; it uses repeat applications of clustering and ML models to generate gene expression profiles - it identifies distinct cell groups and a weighted list of feature genes for each group

> What is the architecture/algorithm of the model?
>> ML [logistic regression, random forest, Gaussian process classification, support vector machine and decision tree] and 5-fold CV

### Import modules

In [3]:
import sys
import anndata
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import numpy as np
import pandas as pd
import scanpy as sc
import numpy.random as random


from umap import UMAP
import warnings; warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import SCCAF as sccaf
from SCCAF import SCCAF_assessment, plot_roc


ImportError: cannot import name 'pkg_metadata' from 'scanpy._compat' (/Users/mairi.mcclean/mambaforge/envs/sccaf_local/lib/python3.10/site-packages/scanpy/_compat.py)

### Read in adata objects

##### adata object

In [ ]:
adata = sc.read_h5ad('/Volumes/LaCie/data_lake/Mairi_example/processed_files/scvi/pre_sccaf/Cai_healthy_scRNA_PBMC_mm2303016_pre_sccaf.h5ad')
adata

In [ ]:
adata.obs

#### caiy_healthy object

In [ ]:
caiy_healthy = sc.read_h5ad('/Volumes/LaCie/data_lake/Mairi_example/processed_files/scvi/pre_sccaf/Cai_healthy_scRNA_PBMC_mm2303016_pre_sccaf_caiy_healthy.h5ad')

#### Clustering

In [ ]:
y_prob, y_pred, y_test, clf, cvsm, acc = SCCAF_assessment(adata.X, adata.obs['leiden'], n = 100)


In [ ]:
sc.tl.leiden(adata, resolution = 0.7, random_state = 1786)

In [ ]:
plot_roc(y_prob, y_test, clf, cvsm = cvsm, acc = acc)
plt.show()

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['leiden', 'status', 'CD74'], size = 0.8, legend_fontsize = 5, legend_loc = 'on data')

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['leiden', 'status', 'tissue', 'ADH7', 'CDH1', 'CD74', 'CD3E', 'MUC20', 'DUSP4', 'FOXJ1', 'MUC1', 'FOXI1'], size = 1, legend_fontsize = 5)

### Export clustered object

In [ ]:
adata
caiy_healthy

In [ ]:
# Making a hybrid anndata object using sections from both original anndata object and the cai_tb_gex object
adata_export = anndata.AnnData(X = caiy_healthy.X, var = caiy_healthy.var, obs = adata.obs, uns = adata.uns, obsm = adata.obsm, layers = caiy_healthy.layers, obsp = adata.obsp)
adata_export

In [ ]:
adata_export.write('/Volumes/Lacie/data_lake/Mairi_example/processed_files/scvi/post_sccaf/CaiY_healthy_scRNA_PBMC_mm230316_scVI-clustered.raw.h5ad')
